In [1]:
import numpy as np # linear algebra
import tensorflow as tf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from collections import Counter
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current 
filename = "/kaggle/input/worldnews-on-reddit/reddit_worldnews_start_to_2016-11-22.csv"
df = pd.read_csv(filename, usecols=['title'], dtype={'title': 'str'}).pop('title')
vocabulary = Counter()
i = 0
for title in df:
    i = i + 1
    if (i == 10):
        break
    vocabulary.update(title.split())
print(vocabulary)

Counter({'to': 3, 'the': 2, 'Scores': 1, 'killed': 1, 'in': 1, 'Pakistan': 1, 'clashes': 1, 'Japan': 1, 'resumes': 1, 'refuelling': 1, 'mission': 1, 'US': 1, 'presses': 1, 'Egypt': 1, 'on': 1, 'Gaza': 1, 'border': 1, 'Jump-start': 1, 'economy:': 1, 'Give': 1, 'health': 1, 'care': 1, 'all': 1, 'Council': 1, 'of': 1, 'Europe': 1, 'bashes': 1, 'EU&UN': 1, 'terror': 1, 'blacklist': 1, 'Hay': 1, 'presto!': 1, 'Farmer': 1, 'unveils': 1, 'illegal': 1, 'mock-Tudor': 1, 'castle': 1, 'he': 1, 'tried': 1, 'hide': 1, 'behind': 1, '40ft': 1, 'hay': 1, 'bales': 1, 'Strikes,': 1, 'Protests': 1, 'and': 1, 'Gridlock': 1, 'at': 1, 'Poland-Ukraine': 1, 'Border': 1, 'The': 1, 'U.N.': 1, 'Mismanagement': 1, 'Program': 1, 'Nicolas': 1, 'Sarkozy': 1, 'threatens': 1, 'sue': 1, 'Ryanair': 1})
